# <center>HW 3: HARSHITA MAHESH HIREMATH </center>

In [5]:
# !pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_colwidth', None)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [6]:
def getProducts(page_url, domain_name):

    results = {'name': [],'description':[],'price':[]}  # variable to hold all records
    
    # Add your code
    
    driver = webdriver.Safari()
    limit = 100

    while page_url and limit > 0:
        driver.get(page_url)

        # Give the page some time to load (you can adjust this as needed)
        time.sleep(2)

        # Get the page source after waiting
        page_source = driver.page_source

        soup = BeautifulSoup(page_source, 'html.parser')
        products = soup.find_all('div', class_='details clearfix')

        for product in products:
            tag = product.find('a')
            name = tag.get('data-product-title')
            description = product.find('span', class_='productDescriptionFull').text.strip()
            cta_price = product.find('a', class_='productPrice')
            if cta_price:
                price_span = cta_price.find('span', class_='currency').find_next('span')
                price = '$' + price_span.text
            results['name'].append(name)
            results['description'].append(description)
            results['price'].append(price)
            limit -= 1

        next_link = soup.find('a', class_='next colorLink')
        if next_link:
            page_url = domain_name + next_link['href']
        else:
            page_url = None

    driver.quit()
    return pd.DataFrame(results)

In [7]:
def getFilteredProducts(driver, page_url, domain_name, category='FlexScan'):
    
    results = {'name': [], 'description': [], 'price': []}

    # Add your code
    driver.get(page_url)

    # Locate the link in the left panel based on the provided category
    category_link = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, category))
    )
    # Click on the category link
    category_link.click()   

    time.sleep(2)

    while page_url:
        page_source = driver.page_source

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')

        # Locate the product elements based on their HTML structure
        products = soup.find_all('div', class_='details clearfix')

        # Extract and store product information
        for product in products:
            tag = product.find('a')
            name = tag.get('data-product-title')
            description = product.find('span', class_='productDescriptionFull').text.strip()
            cta_price = product.find('a', class_='productPrice')
            if cta_price:
                price_span = cta_price.find('span', class_='currency').find_next('span')
                price = '$' + price_span.text
            results['name'].append(name)
            results['description'].append(description)
            results['price'].append(price)

        next_link = soup.find('a', class_='next colorLink')
        if next_link:
            page_url = domain_name + next_link['href']
        else:
            page_url = None


    return pd.DataFrame(results)

In [8]:
# best practice to test your class
# if your script is exported as a module,
# the following part is ignored
# this is equivalent to main() in Java

if __name__ == "__main__":

    print("========Test Q1=========")
    domain_name = 'http://www.pricegrabber.com'
    page_url = 'http://www.pricegrabber.com/computer-monitors/browse/'

    products = getProducts(page_url, domain_name)
    print(len(products))
    print(products.head(5))

    print("\n")
    print("========Test Q2=========")
    
    driver = webdriver.Safari()
    
    driver.implicitly_wait(3)  # set implict wait

    domain_name = 'http://www.pricegrabber.com'
    page_url = 'http://www.pricegrabber.com/computer-monitors/browse/'

    products = getFilteredProducts(driver, page_url, domain_name,
                                   category ='Sun')

    driver.quit()
    print(products)

========Test Q1=========
100
                                                                          name  \
0                                              Legion USB-C 95W AC Adapter(UL)   
1                                              ThinkVision MC60 Monitor Webcam   
2                                        Mobile Pixels TRIO Max 14" LCD 2 Pack   
3  Apple 27" Studio Display (Nano-Texture Glass, VESA Mount Adapter) MMYX3LL/A   
4                             ViewSonic Used VA951S 19" 5:4 IPS Monitor VA951S   

                                                                                                                                                                                                                                                   description  \
0                              Lenovo USB-C 95W AC Adapter offers fast and efficient charging. It is your perfect replacement or spare power adapter for your Lenovo laptops. Lenovo AC adapters are compact, energy efficie